## Guider mode movies
This is intended to unpack the rawStamps from LSSTCAM guider mode FITS files and 
create a movie of all 16 stamps for all 8 guider CCDs. \
Craig Lage - 12-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
import matplotlib.colors as colors
from lsst.summit.utils import getQuantiles
import astropy.visualization as vis

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

In [ ]:
# Set the scaling
autoscale = True
# Scale to use if autoscale = False
vmin = 14500
vmax = 15000

# Now build the frames for the movie

In [ ]:
dayObs = 20250417
seqNum = 599
expId = int(f"{dayObs}{seqNum:05d}")

# Get one CCD to know nStamps
raft = 'R00'
ccd = 'SG0'
filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)

# This defines the plot locations for the 8 CCDs
# Each entry is [raft, ccd, i0, j0, di, dj, rot]
config = [['R00', 'SG0', 20, 9, -1, -1, True], ['R00', 'SG1', 12, 0, -1, 1, False], 
          ['R04', 'SG0', 11, 20, 1, -1, False], ['R04', 'SG1', 20, 12, -1, -1, True], 
          ['R40', 'SG0', 9, 0, -1, 1, False], ['R40', 'SG1', 0, 8, 1, 1, True],
          ['R44', 'SG0', 0, 11, 1, 1, True], ['R44', 'SG1', 8, 20, 1, -1, False]]

dirName = f"/home/c/cslage/u/Guider_Mode/LSSTCam_movie_{expId}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}.mp4"
print(movieName)
# Build the individual frames
fig = plt.figure(figsize=(10,10))
for n in [1]:#range(1, nStamps+1):
    axs = fig.subplots(21,21)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    # Clear the axes and plot frames
    for i in range(21):
        for j in range(21):
            axs[i][j].axis('off')
            axs[i][j].set_xticks([])
            axs[i][j].set_yticks([])
    for [raft, ccd, i0, j0, di, dj, rot] in config:
        filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
        
        rp = ResourcePath(filename)
        with rp.open(mode="rb") as f:
            hdu_list = fits.open(f)
        [roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
        hduNum = 2 * n + 1
        hdrn = hdu_list[hduNum].header
        timestamp = hdrn['STMPTIME']
        image_out = unpackStamps(hduNum)
        i = i0; j = j0
        # Set the CCD titles
        if rot:
            if dj < 0:
                axs[i0 + 3 * di][j0 - 1].set_title(f"{raft}_{ccd}", fontsize=12, rotation='vertical',x=-0.3,y=0.5)
            else:
                axs[i0 + 3 * di][j0].set_title(f"{raft}_{ccd}", fontsize=12, rotation='vertical',x=-0.4,y=-0.5)
        else:
            if di < 0:
                axs[i0 - 1][j0 + 3 * dj].set_title(f"{raft}_{ccd}", fontsize=12, loc='center')
            else:
                axs[i0][j0 + 3 * dj].set_title(f"{raft}_{ccd}", fontsize=12, loc='center')
        # Now plot the data
        for seg in range(16):
            #print(raft, ccd, seg, i, j)
            arr = image_out[seg]
            #quantiles = getQuantiles(arr, 128)
            #norm = colors.BoundaryNorm(quantiles, 128)
            percentile = 1.0
            #interval = vis.PercentileInterval(percentile)
            #norm = vis.ImageNormalize(arr, interval=interval, stretch=vis.LogStretch(a=1))
            med = np.median(arr)
            vmin = med * (1.0 - percentile / 100.0)
            vmax = med * (1.0 + percentile / 100.0)
            
            if rot:
                arr = np.transpose(arr)
            im = axs[i][j].imshow(arr, interpolation='nearest', 
                                  origin='lower', cmap='Greys', vmin=vmin, vmax=vmax)#norm=norm)

            #axs[i][j].text(roiRows/2.0, roiCols/2.0,f"{seg}") # For segment check
            if rot:
                if seg == 7:
                    j += dj
                    i = i0
                else:
                    i += di
            else:
                if seg == 7:
                    i += di
                    j = j0
                else:
                    j += dj
    plt.suptitle(f"Guider mode {expId}, \n Frame {n+1} {timestamp}", fontsize=18) 
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()